In [ ]:
import numpy as np

from src.sampling.distributions import Distribution
from src.items.base import Item, create_item
from src.item_category.base import ItemCategory


items: list[Item] = [
    create_item(i) for i in range(1, 11)
]

category = ItemCategory(
    service_category_id=1,
    likelihood=0.4,
    quantity_distribution=Distribution(
        distribution_type="longtail",
        lower_bound=1,
        upper_bound=5,
        int_or_float="int"
    ),
    items=items
)

In [17]:
category.sample_items()

[{'service_id': 10, 'price': 74.1, 'quantity': 1, 'variant': None},
 {'service_id': 4, 'price': 38.13, 'quantity': 2, 'variant': None},
 {'service_id': 8, 'price': 85.37, 'quantity': 6, 'variant': None},
 {'service_id': 6, 'price': 55.73, 'quantity': 15, 'variant': None}]